In [ ]:
from pyspark.sql import SparkSession

In [ ]:
# Criação da sessão Spark
spark = SparkSession.builder.appName("EMR_LogAnalysis").getOrCreate()

In [ ]:
# Lê o arquivo de log do S3
log_file = "s3://aws-emr-studio-432600844323-us-east-1/1736180251020/e-3K4ZJDEXJZTKO382BVWX4D3PO/access.log"  # Substitua pelo caminho real
logs_rdd = spark.sparkContext.textFile(log_file)

In [ ]:
# Processa cada linha do log
ip_counts = (
    logs_rdd.map(
        lambda line: line.split(" ")[0])  # Extrai o IP (primeiro campo)
    .map(lambda ip: (ip, 1))  # Cria pares (IP, 1)
    .reduceByKey(lambda a, b: a + b)  # Soma os acessos por IP
)

In [ ]:
# Converte para DataFrame para visualização
ip_counts_df = ip_counts.toDF(["IP", "Count"]).orderBy("Count", ascending=False)

In [ ]:
# Salva os resultados no S3
output_path = "s3://aws-emr-studio-432600844323-us-east-1/1736180251020/e-3K4ZJDEXJZTKO382BVWX4D3PO/ip_counts"
ip_counts_df.write.csv(output_path, header=True)

In [ ]:
# Mostra os resultados (limitado para evitar grandes volumes)
ip_counts_df.show(10)

In [ ]:
# Finaliza a sessão Spark
spark.stop()